# PetrovDS
Решение с использованием Decision Tree

## Библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Чтение данных

In [2]:
train = pd.read_csv("PD-data-train.csv", sep=";", index_col="record_id")
test = pd.read_csv("PD-data-test.csv", sep=";", index_col="record_id")
desc = pd.read_csv("PD-data-desc.csv", sep=";")

train_raw = train.copy()
test_raw = test.copy()

## OneHotEncoding количества сотрудников

In [3]:
train.ul_staff_range = train.ul_staff_range.map({'[1-100]': 'stuff_small','(100-500]': 'stuff_mid','> 500': 'stuff_large'})
train = train.join(pd.get_dummies(train.ul_staff_range))
train = train.drop(["ul_staff_range"], axis=1)

test.ul_staff_range = test.ul_staff_range.map({'[1-100]': 'stuff_small','(100-500]': 'stuff_mid','> 500': 'stuff_large'})
test = test.join(pd.get_dummies(test.ul_staff_range))
test = test.drop(["ul_staff_range"], axis=1)

## Разбиваем на бакеты ul_founders_cnt

In [4]:
def founders_bucket(x):
    if x < 4:
        return "fnd_" + str(x)
    if 3 < x < 11:
        return "fnd_4_10"
    if 11 < x < 51:
        return "fnd_11_50"
    if x > 500:
        "fnd_" + str(x)
    return "fnd_50"

train = train.join(pd.get_dummies(train.ul_founders_cnt.map(lambda x: founders_bucket(x))))
train = train.drop(["ul_founders_cnt"], axis=1)

test = test.join(pd.get_dummies(test.ul_founders_cnt.map(lambda x: founders_bucket(x))))
test = test.drop(["ul_founders_cnt"], axis=1)

## Уже не помню зачем это, но, видимо, что-то очень важное

In [5]:
train.ar_other_profit_and_losses = train.ar_other_profit_and_losses.map(lambda x: 1 if x != 0 else 0)
test.ar_other_profit_and_losses = test.ar_other_profit_and_losses.map(lambda x: 1 if x != 0 else 0)

## Убираем сразу один лишний признак

In [6]:
# ul_branch_cnt на Test имеет только значения 0
train = train.drop(["ul_branch_cnt"], axis=1)
test = test.drop(["ul_branch_cnt"], axis=1)

## Функция, которая возвращает признаки по заданному порогу корреляции

In [13]:
def drop_corr(df, T):
    cr = df.corr().dropna(how="all").dropna(axis=1, how="all")
    return [col for ind, col in enumerate(cr) if ((cr.iloc[ind,ind+1:] > T) | (cr.iloc[ind,ind+1:] < -T)).any()] + list(set(df.columns) - set(cr.columns))

## Сама модель, в которой и fit, и predict

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

def DT(df, t_df):
    y = df.default_12m
    X = df.drop(["default_12m"], axis=1)

    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    t_df = scaler.transform(t_df)
    
    clf = DecisionTreeClassifier(max_depth=None, min_samples_split=300)
    print(np.array(cross_val_score(clf, X, y, cv=10)).mean())
    
    clf.fit(X, y)
    return clf.predict_proba(t_df)

## Создаём копию, дабы не поцарапать первоначальные данные

In [8]:
tf_train = train.copy()
tf_test = test.copy()

## Делим на два DF: один с заполненными столбцами, второй с пропущенными столбцами

In [9]:
train_na = tf_train[tf_train["ar_sale_cost"].isnull()].dropna(axis=1)
train_fl = tf_train.dropna()

In [10]:
test_na = tf_test[tf_test["ar_sale_cost"].isnull()].dropna(axis=1)
test_fl = tf_test.dropna()

## Удаляем коррелирующие столбцы. Порог 0.5 получен подбором

In [14]:
drop_col_na = drop_corr(train_na, .5)
drop_col_fl = drop_corr(train_fl, .5)

In [15]:
train_na = train_na.drop(drop_col_na, axis=1)
test_na = test_na.drop(drop_col_na, axis=1)
train_fl = train_fl.drop(drop_col_fl, axis=1)
test_fl = test_fl.drop(drop_col_fl, axis=1)

## fit-predict

In [16]:
y_pred_fl = DT(train_fl, test_fl)
y_pred_na = DT(train_na, test_na)

0.9521159420289855
0.9142952683643528


## cutoff выбирается как среднее минус одна сотая. 0.01 получена подгоном С:

In [37]:
T_c = np.array(list(y_pred_na[:,0]) + list(y_pred_fl[:,0])).mean() - .01
T_c

0.8987642980866306

## Так надо. Правда

In [33]:
T_fl = T_c
T_na = T_c

## Определяем классы по cuttoff и загоняем ответы в csv

In [34]:
res_na = pd.DataFrame(zip(list(test_na.index), [0 if i[0] > T_fl else 1 for i in np.array(list(y_pred_na))]), columns=["id", "predict"])
res_fl = pd.DataFrame(zip(list(test_fl.index), [0 if i[0] > T_na else 1 for i in np.array(list(y_pred_fl))]), columns=["id", "predict"])

In [35]:
res = pd.concat([res_na, res_fl])

In [36]:
test.drop(test.columns, axis=1).merge(res, left_index=True, right_on="id").to_csv('PD-submit.csv',index=False, sep=';')